In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df_social = pd.read_csv('../Data/Preprocessed_data/SocialMediaReviews.csv')
df_social.drop(columns=['ViewCount'],inplace=True)
df_social.dropna(inplace=True)
print(len(df_social))


60285


In [2]:
documents = df_social['Review'][-500:]
len(documents)

500

Preprocessing

In [3]:
from multiprocessing import Pool
from nltk import word_tokenize

documents = df_social['Review'][-20:]
stopwords = set(open('../Model/vietnamese-stopwords_edit.txt', encoding='utf-8').read().split('\n')[:-1])
puct_set = set([c for c in '!"#$%&\'()*+,./:;<=>?@[\\]^`{|}~'])

def generateBigram(review):
    words = review.split()
    if len(words)==1:
        return ''
    bigrams = [words[i]+'_'+words[i+1] for i in range(0,len(words)-1)]
    return ' '.join(bigrams)

def removeRedundant(text,redundantSet):
    words = text.split()
    for i in range(0,len(words)):
        if words[i].count('_') == 0 and (words[i] in redundantSet or words[i].isdigit()):
            words[i] = ''
        else:
            sub_words = words[i].split('_')
            if any(w in redundantSet or w.isdigit() for w in sub_words):
                words[i] = ''
    words = [w for w in words if w != '']
    words = ' '.join(words)
    return words


def preprocessing(text):
    text = ' '.join(word_tokenize(text))
    text = text.lower()
    text = ' '.join(text.split())
    text = text + generateBigram(text)
    text = removeRedundant(text,puct_set | stopwords)
    return text

clean_documents =  documents.apply(preprocessing)
# pool = Pool(10)
# clean_documents = pool.map(preprocessing,documents)
# pool.terminate()


In [7]:
clean_documents

60802    reset tai thi thoảng kết nối lỗi buds zflip hà...
60803                  việt nam toàn lỗi việt_nam nam_toàn
60804    kĩ à. đánh đổi hai sim vật lý cắt bớt ram đi m...
60805              hàn clip sim r kìa .bản_hàn sim_r r_kìa
60806                sim vật lí esim .1_sim sim_vật vật_lí
60807                  cắt bớt ram nnao bạncắt_bớt bớt_ram
60808                                           chẳng biệt
60809    may thang lay dt noi cau laido b oimay_thang l...
60810    samsung học ip công adr 8gb ram đuồi hệ hành k...
60811    xài s23 ultra xài hơi tiếc flashship lẻ đc ram...
60812    tất nhiên xài ko lẽ lăn chết giá tốt 8gb ram đ...
60813    s23u 8/256gb hơi tiếc flagship kẹo s23u_8/256g...
60814    ​ kuboboanimehmm 8gb ram chạy đa nhiệm dex gam...
60815                                                     
60816    global hỗ trợ samsung pay t ngân khuyến khích ...
60817       google pay thay vấn google_pay pay_thay vấn_đề
60818      hàn sản xuất hàn quốc hàn_sản sản_xuất hàn_qu

Feature Extraction = LSA = TruncatedSVD(TF-IDF)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD,PCA
vectorizer =  TfidfVectorizer(token_pattern="S+",min_df=2)
vectors = vectorizer.fit_transform(clean_documents)

ValueError: empty vocabulary; perhaps the documents only contain stop words

<class 'pandas.core.series.Series'>
Index: 20 entries, 60802 to 60821
Series name: Review
Non-Null Count  Dtype 
--------------  ----- 
20 non-null     object
dtypes: object(1)
memory usage: 876.0+ bytes


In [ ]:
print("Tfidf shape:"+str(vectors.shape))
svd = TruncatedSVD(n_components=100,n_iter=10)
svd_vectors=svd.fit_transform(vectors)
print("Document 1's Vector : ")
print(svd_vectors[0])